In [3]:
import keras

keras.__version__

'2.10.0'

In [7]:
from tensorflow.keras.utils import image_dataset_from_directory
from pathlib import Path
import tensorflow as tf

dataset_path = Path("C:/Users/nutae/.cache/kagglehub/datasets/birdy654/cifake-real-and-ai-generated-synthetic-images/versions/3")

train_dataset = image_dataset_from_directory(
    dataset_path / 'train',
    image_size=(224, 224),
    batch_size=32,
    label_mode='binary',
    seed=0,
    validation_split=0.2,
    subset='training'
)

validation_dataset = image_dataset_from_directory(
    dataset_path / 'train',
    image_size=(224, 224),
    batch_size=32,
    label_mode='binary',
    seed=0,
    validation_split=0.2,
    subset='validation'
)

# validation_split = 0.2
# total_batches = tf.data.experimental.cardinality(train_dataset).numpy()
# val_batches = int(validation_split * total_batches)

# train_dataset = train_dataset.skip(val_batches)
# validation_dataset = train_dataset.take(val_batches)


Found 100000 files belonging to 2 classes.
Using 80000 files for training.
Found 100000 files belonging to 2 classes.
Using 20000 files for validation.


In [8]:
train_dataset

<BatchDataset element_spec=(TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None, 1), dtype=tf.float32, name=None))>

In [9]:
validation_dataset

<BatchDataset element_spec=(TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None, 1), dtype=tf.float32, name=None))>

In [10]:
from tensorflow.keras.applications.resnet import preprocess_input
from tensorflow.data import AUTOTUNE

train_dataset = train_dataset.map(lambda x, y: (preprocess_input(x), y), num_parallel_calls=AUTOTUNE)
validation_dataset = validation_dataset.map(lambda x, y: (preprocess_input(x), y), num_parallel_calls=AUTOTUNE)

# caching/prefetching for better performance
# train_dataset = train_dataset.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
# validation_dataset = validation_dataset.cache().prefetch(buffer_size=AUTOTUNE)

In [11]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras import layers, models

base_model = ResNet50(weights="imagenet", include_top=False, input_shape=(224, 224, 3))
base_model.trainable = False 

model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(256, activation="relu"),
    layers.Dropout(0.5),
    layers.Dense(1, activation="sigmoid")
])

In [12]:
model.compile(
    optimizer="adam",
    loss="binary_crossentropy",
    metrics=["accuracy"]
)

In [ ]:
history = model.fit(
    train_dataset,
    validation_data=validation_dataset,
    epochs=5
)